# Altair example NAV dagpenger

In [2]:
import deetly
import os
import pandas as pd
import pydeck
import geopandas as gpd
from getpass import getpass

import altair as alt

In [3]:
MAPBOX_TOKEN = os.environ['MAPBOX_TOKEN'] 

In [4]:
EXCEL_URL_DAGPENGER = 'https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/soknader-om-dagpenger/_/attachment/download/39af8c38-f8fc-4810-9242-979658ecc510:04375076c6a88c8cbe3567397f7b1ecb18bc7f1a/170320%20-%20S%C3%B8knader_om_dagpenger.xlsx'

In [5]:
EXCEL_URL_SYKEFRAVAER = 'https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/sykefravaerstilfeller-pavist-eller-mistanke-om-smitte-av-koronaviruset/_/attachment/download/eb4968b7-f8f0-4664-826a-4c1a13f8ffb3:dfbd16c710e32edebcee4dcb6838e572ed89adc0/190320%20-%20Sykmeldinger%20-%20corona%20(dag%20og%20uke).xlsx'

In [6]:
MAP_URL = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/2020/fylker_2020_s_geojson.json'

In [7]:
xl_sykefravaer = pd.ExcelFile(EXCEL_URL_SYKEFRAVAER)
xl_sykefravaer.sheet_names

['Per dag', 'Fylke dag', 'Fylke uke', 'Diagnoser', 'Om statistikken']

In [8]:
df = pd.read_excel(EXCEL_URL_SYKEFRAVAER,sheet_name='Fylke dag', header=6, encoding="latin-1")
df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
df.rename(columns={"Unnamed: 1": "Time"}, inplace=True)
df.drop([0, 1],inplace=True)
df.head(5)

,Time,Hele landet,Oslo,Rogaland,Møre og Romsdal,Nordland,Viken,Innlandet,Vestfold og Telemark,Agder,Vestland,Trøndelag,Troms og Finnmark,Ukjent
2,2020-03-17 00:00:00,2757,543,304,98,57,669,143,202,136,307,186,94,18
3,2020-03-16 00:00:00,5476,981,561,170,106,1337,304,416,316,787,293,185,20
4,2020-03-15 00:00:00,442,84,43,13,7,100,22,29,18,85,20,18,*
5,2020-03-14 00:00:00,460,77,43,11,5,120,17,27,26,106,6,16,6
6,2020-03-13 00:00:00,4098,584,467,138,186,1034,172,357,183,507,298,155,17


In [9]:
import time
data = pd.melt(df, id_vars=['Time'])
data.columns=['time','name','value']
data['day'] = data['time'].apply(lambda x: int(1584399600 - int(time.mktime(x.timetuple())))/86400)
data['fylke_navn'] = data['name'].apply(lambda x: str(x).upper().strip())
data = data[~data['fylke_navn'].isin(['HELE LANDET', 'UKJENT'])]
data['value']= data['value'].replace('*', 0) 
data

,time,name,value,day,fylke_navn
10,2020-03-17,Oslo,543,0.0,OSLO
11,2020-03-16,Oslo,981,1.0,OSLO
12,2020-03-15,Oslo,84,2.0,OSLO
13,2020-03-14,Oslo,77,3.0,OSLO
14,2020-03-13,Oslo,584,4.0,OSLO
...,...,...,...,...,...
115,2020-03-12,Troms og Finnmark,55,5.0,TROMS OG FINNMARK
116,2020-03-11,Troms og Finnmark,45,6.0,TROMS OG FINNMARK
117,2020-03-10,Troms og Finnmark,12,7.0,TROMS OG FINNMARK
118,2020-03-09,Troms og Finnmark,21,8.0,TROMS OG FINNMARK


In [10]:
df_gpd = gpd.read_file(MAP_URL)
df_gpd

,id,fylke_navn,fylke_nummer,area_fylke,geometry
0,0,AGDER,42,2.527201,"POLYGON ((6.42634 58.28968, 6.56033 58.32740, ..."
1,1,INNLANDET,34,8.758690,"POLYGON ((8.25170 61.07394, 8.14393 61.10959, ..."
2,2,MØRE OG ROMSDAL,15,2.477828,"MULTIPOLYGON (((5.78037 62.21080, 5.60243 62.1..."
3,3,NORDLAND,18,7.697804,"MULTIPOLYGON (((11.99575 65.63535, 11.88899 65..."
4,4,OSLO,03,0.072515,"POLYGON ((10.76292 59.82888, 10.75187 59.90816..."
5,5,ROGALAND,11,1.449727,"MULTIPOLYGON (((5.31375 59.17696, 5.19362 59.1..."
6,6,TROMS OG FINNMARK,54,17.315632,"MULTIPOLYGON (((16.43876 68.50629, 16.51196 68..."
7,7,TRØNDELAG,50,7.595582,"MULTIPOLYGON (((8.77547 63.80562, 8.83178 63.6..."
8,8,VESTFOLD OG TELEMARK,38,2.753867,"MULTIPOLYGON (((10.38534 59.53393, 10.23186 59..."
9,9,VESTLAND,46,5.513065,"MULTIPOLYGON (((5.45287 59.49821, 5.30430 59.4..."


In [11]:
df_fylker = df_gpd[['fylke_navn','fylke_nummer']].drop_duplicates()
df_fylker.columns=['name','id']
df_fylker

,name,id
0,AGDER,42
1,INNLANDET,34
2,MØRE OG ROMSDAL,15
3,NORDLAND,18
4,OSLO,03
5,ROGALAND,11
6,TROMS OG FINNMARK,54
7,TRØNDELAG,50
8,VESTFOLD OG TELEMARK,38
9,VESTLAND,46


In [12]:
data_today = data[data['time']=='2020-03-17']
data_today

,time,name,value,day,fylke_navn
10,2020-03-17,Oslo,543,0.0,OSLO
20,2020-03-17,Rogaland,304,0.0,ROGALAND
30,2020-03-17,Møre og Romsdal,98,0.0,MØRE OG ROMSDAL
40,2020-03-17,Nordland,57,0.0,NORDLAND
50,2020-03-17,Viken,669,0.0,VIKEN
60,2020-03-17,Innlandet,143,0.0,INNLANDET
70,2020-03-17,Vestfold og Telemark,202,0.0,VESTFOLD OG TELEMARK
80,2020-03-17,Agder,136,0.0,AGDER
90,2020-03-17,Vestland,307,0.0,VESTLAND
100,2020-03-17,Trøndelag,186,0.0,TRØNDELAG


In [13]:
data.head(10)

,time,name,value,day,fylke_navn
10,2020-03-17,Oslo,543,0.0,OSLO
11,2020-03-16,Oslo,981,1.0,OSLO
12,2020-03-15,Oslo,84,2.0,OSLO
13,2020-03-14,Oslo,77,3.0,OSLO
14,2020-03-13,Oslo,584,4.0,OSLO
15,2020-03-12,Oslo,368,5.0,OSLO
16,2020-03-11,Oslo,180,6.0,OSLO
17,2020-03-10,Oslo,103,7.0,OSLO
18,2020-03-09,Oslo,102,8.0,OSLO
19,2020-03-08,Oslo,9,9.0,OSLO


In [16]:
map_data = alt.Data(url=MAP_URL, format=alt.DataFormat(property='features',type='json'))

values = alt.pipe(data[['fylke_navn', 'value', 'day']], alt.to_values)

day_slider = alt.binding_range(min=0, max=9, step=1)
slider_selection = alt.selection_single(bind=day_slider, fields=['day'], init={'day': 9}, name='day_')

base = alt.Chart(map_data).mark_geoshape(
).encode(
    color = alt.Color('value:Q'),
).transform_lookup(
    lookup='properties.fylke_navn',
    from_=alt.LookupData(values, 'fylke_navn', ['value', 'day'])
).properties(
    projection={'type': 'mercator'}
).configure_legend(
  orient='top'
).configure_view(
    strokeWidth=0
)

filter_day = base.add_selection(
    slider_selection
).transform_filter(
    slider_selection
)

filter_day

alt.Chart(...)

In [17]:
def getMap(day):
    
    values = alt.pipe(data[['fylke_navn', 'value', 'day']][data['day']==day], alt.to_values)
    
    map_data = alt.Data(url=MAP_URL, format=alt.DataFormat(property='features',type='json'))

    fig = alt.Chart(map_data).mark_geoshape(
    ).encode(
        color = alt.Color('value:Q'),
    ).transform_lookup(
        lookup='properties.fylke_navn',
        from_=alt.LookupData(values, 'fylke_navn', ['value'])
    ).properties(
        projection={'type': 'mercator'}
    ).configure_legend(
      orient='top'
    ).configure_view(
        strokeWidth=0
    )

    return fig

In [18]:
for day in range(0,10):
    getMap(day)
    
getMap(0)

alt.Chart(...)

In [19]:
getMap(1)

alt.Chart(...)

In [20]:
getMap(2)

alt.Chart(...)

In [21]:
getMap(3)

alt.Chart(...)

In [22]:
getMap(4)

alt.Chart(...)

In [23]:
getMap(5)

alt.Chart(...)